### [731\. My Calendar II](https://leetcode.com/problems/my-calendar-ii/)

Difficulty: **Medium**


Implement a `MyCalendarTwo` class to store your events. A new event can be added if adding the event will not cause a **triple** booking.

Your class will have one method, `book(int start, int end)`. Formally, this represents a booking on the half open interval `[start, end)`, the range of real numbers `x` such that `start <= x < end`.

A _triple booking_ happens when **three** events have some non-empty intersection (ie., there is some time that is common to all 3 events.)

For each call to the method `MyCalendar.book`, return `true` if the event can be added to the calendar successfully without causing a **triple** booking. Otherwise, return `false` and do not add the event to the calendar.

Your class will be called like this: `MyCalendar cal = new MyCalendar();` `MyCalendar.book(start, end)`

**Example 1:**

```
MyCalendar();
MyCalendar.book(10, 20); // returns true
MyCalendar.book(50, 60); // returns true
MyCalendar.book(10, 40); // returns true
MyCalendar.book(5, 15); // returns false
MyCalendar.book(5, 10); // returns true
MyCalendar.book(25, 55); // returns true
Explanation: 
The first two events can be booked.  The third event can be double booked.
The fourth event (5, 15) can't be booked, because it would result in a triple booking.
The fifth event (5, 10) can be booked, as it does not use time 10 which is already double booked.
The sixth event (25, 55) can be booked, as the time in [25, 40) will be double booked with the third event;
the time [40, 50) will be single booked, and the time [50, 55) will be double booked with the second event.
```

**Note:**

*   The number of calls to `MyCalendar.book` per test case will be at most `1000`.
*   In calls to `MyCalendar.book(start, end)`, `start` and `end` are integers in the range `[0, 10^9]`.

In [23]:
# Counter/OrderedDict O(N^2*logN) - TLE
# Boundary Counting
from collections import OrderedDict

class MyCalendarTwo:

    def __init__(self):
        self.cnt = OrderedDict()

    def book(self, start: int, end: int) -> bool:
        if start not in self.cnt:
            self.cnt[start] = 1
        else:
            self.cnt[start] += 1 # start is inclusive
        if end not in self.cnt:
            self.cnt[end] = -1
        else:
            self.cnt[end] -= 1 # end is exclusive
        cur_overlap = 0
        for key in sorted(self.cnt.keys()):
            cur_overlap += self.cnt[key]
            if cur_overlap >= 3:
                # book failed, reset counter to previous state
                self.cnt[start] -= 1
                self.cnt[end] += 1
                return False
        return True

In [35]:
# Boundary Counting + Binary Search Insort
# https://leetcode.com/problems/my-calendar-ii/discuss/192388/python-boundary-count-using-bisect
from bisect import insort

class MyCalendarTwo:

    def __init__(self):
        self.cnt = []

    def book(self, start: int, end: int) -> bool:
        if not self.cnt:
            self.cnt += (start,1),
            self.cnt += (end,-1),
            return True
        insort(self.cnt, (start,1))
        insort(self.cnt, (end,-1))
        cur_overlap = 0
        for _, freq in self.cnt:
            cur_overlap += freq
            if cur_overlap == 3:
                # book failed, reset counter to previous state
                self.cnt.remove((start,1))
                self.cnt.remove((end,-1))
                return False
        return True

In [31]:
# https://leetcode.com/problems/my-calendar-ii/discuss/290934/python-bisectinsort-97
from bisect import *

class MyCalendarTwo:

    def __init__(self):
        self.cnt = {}
        self.pos = []

    def book(self, start: int, end: int) -> bool:
        si = bisect_left(self.pos, start)
        ei = bisect_left(self.pos, end)
        if any(self.cnt[self.pos[k]] >= 2 for k in range(si, ei)):
            return False
        if start not in self.cnt:
            c = self.cnt[self.pos[si-1]] if si>=1 else 0
            if c >= 2: return False
            # insert one element at index i
            self.pos[si:si] = [start]
            self.cnt[start] = c
            ei += 1
        if end not in self.cnt:
            self.pos[ei:ei] = [end]
            self.cnt[end] = self.cnt[self.pos[ei-1]]
        for k in range(si,ei):
            self.cnt[self.pos[k]] += 1
        return True

In [36]:
# Your MyCalendarTwo object will be instantiated and called as such:
obj = MyCalendarTwo()
print(obj.book(10,20))
print(obj.book(50,60))
print(obj.book(10,40))
print(obj.book(5, 15))
print(obj.book(5, 10))
print(obj.book(25, 55))

True
True
True
False
True
True
